# exploration_useful_cols.ipynb

This notebook is used to explore the tables and columns of the database and to filter out tables / columns / rows that carry no relevant information.  
The goal is to reduce the amount of data that needs to be processed and to reduce the complexity of the data.  
The resulting alterations are collected in the sql script dataset/sql/initial_cleanup.sql

In [9]:
import psycopg2

In [35]:
# get connection
conn = psycopg2.connect(
    host='localhost',
    database='cadets_e3',
    user='rosendahl',
)
# get cursor
cur = conn.cursor()


In [38]:
def find_useless_columns(table_name):
    """
    Finds all columns in a table that have only one distinct values or are always NULL.
    :return: a list of column names
    """
    # select column names from table
    cur.execute('select column_name from information_schema.columns where table_name = %s', (table_name,))
    column_names_r = cur.fetchall()

    # convert to list of strings
    col_names = [row[0] for row in column_names_r]
    no_variance = []

    for col in col_names:
        cur.execute(f'select count(distinct {col}) from {table_name}')
        distinct_count = cur.fetchone()[0]
        if distinct_count <= 1:
            no_variance.append(col)
    
    return no_variance

## get all tables
Note: this reuses code from database_stats.ipynb

In [40]:
query = '''
select table_name
from information_schema.tables
where table_schema = 'public'
'''

cur.execute(query)
tables_result = cur.fetchall()

In [41]:
# transform into list
tables = [table[0] for table in tables_result]
# discard event partition tables
tables = [table for table in tables if not table.startswith('event_p')]

display(tables)

['event',
 'fileobject',
 'netflowobject',
 'node_uuids',
 'principal',
 'srcsinkobject',
 'subject',
 'unnamedpipeobject']

## primary keys
Since the tables do not contain keys, we need to find a way to uniquely identify rows.  
Specifically the tables __event__ and __subject__ are of interest.

In [12]:
query = '''
select count(*), count(distinct e.uuid), count(distinct (e.uuid, e.sequence_long)), count(distinct (e.uuid, e.timestampnanos))
from event e;
'''

cur.execute(query)
result = cur.fetchone()

In [13]:
no_events, no_dist_uuids, no_dist_uuid_seq, no_dist_uuid_ts = result

print(f'number of events: {no_events}')
print(f'number of distinct uuids: {no_dist_uuids}')
print(f'number of distinct uuids and sequence_long: {no_dist_uuid_seq}')
print(f'number of distinct uuids and timestampnanos: {no_dist_uuid_ts}')

number of events: 41350895
number of distinct uuids: 20606917
number of distinct uuids and sequence_long: 20606917
number of distinct uuids and timestampnanos: 41350895


The uuids are not unique, this is probably due to the fact that cadets crashed during attacks.  
However, the combination of uuid and sequence_long is unique.

With this info, lets check if the same holds for the __subject__ table.

In [14]:
query = '''
select count(*), count(distinct uuid)
from subject;
'''

cur.execute(query)
result = cur.fetchone()

In [15]:
no_subjects, no_dist_subj_uuids = result

print(f'number of subjects: {no_subjects}')
print(f'number of distinct uuids: {no_dist_subj_uuids}')

number of subjects: 224629
number of distinct uuids: 224629


The uuids in the __subject__ table are unique.  

## table _event_import_
The table __event_import__ looks very similar to the table __event__.
Lets check if the table __event_import__ is a subset of the table __event__.

In [17]:
query = '''
SELECT COUNT(*)
FROM event_import ei
LEFT JOIN event e ON ei.uuid = e.uuid AND ei.timestampnanos = e.timestampnanos
WHERE e.uuid IS NULL;
'''

cur.execute(query)

print(f'number of events in event_import that are not in event: {cur.fetchone()[0]}')

number of events in event_import that are not in event: 0


The table __event_import__ is a subset of the table __event__.  
=> drop entire table

## table _host_
The ground truth says that there is only one host in the dataset, yet the table __host__ contains multiple rows.  
Verify that all the rows are the same.

In [28]:
# Omit the columns 'line_no' and 'line' as they do not contain relevant information
query = '''
select count(distinct (uuid, hostname, hostidentifiers, osdetails, hosttype, interfaces))
from host;
'''

cur.execute(query)

print(f'number of distinct hosts: {cur.fetchone()[0]}')

number of distinct hosts: 1


As there is only one distinct host, the variance in the data is 0.  
The data may be useful for interpretation, but it is irrelevant for training.  
=> drop entire table 

## table _principal_
At first glance, the table seems to contain duplicate entries, only differing in the columns 'line_no' and 'line'.  
Find duplicate entries.

In [31]:
query = '''
select count(*), count(distinct username_string)
from principal;
'''

cur.execute(query)
principal_count_result = cur.fetchone()

print(f'number of principals: {principal_count_result[0]}')
print(f'number of distinct principals: {principal_count_result[1]}')

number of principals: 63
number of distinct principals: 22


=> drop duplicate entries in the table __principal__.

## columns _line_no_ and _line_
The columns _line_no_ and _line_ are present in (almost) all tables.  
They are a result of the import and are not relevant for training.  
=> drop columns _line_no_ and _line_ from all tables


## columns with no or only one distinct value
columns with variance 0 (== one distinct value) are not relevant for training.

In [42]:
useless_cols = {}

for table in tables:
    useless_cols[table] = find_useless_columns(table)

In [43]:
display(useless_cols)

{'event': ['predicateobject2path',
  'predicateobjectpath',
  'size',
  'predicateobject2',
  'predicateobject',
  'subject',
  'name',
  'parameters',
  'hostid',
  'location',
  'programpoint',
  'properties_map_host'],
 'fileobject': ['filedescriptor',
  'localprincipal',
  'size',
  'peinfo',
  'hashes',
  'baseobject_hostid',
  'baseobject_permission',
  'baseobject_epoch'],
 'netflowobject': ['ipprotocol',
  'filedescriptor',
  'baseobject_hostid',
  'baseobject_permission',
  'baseobject_epoch'],
 'node_uuids': [],
 'principal': ['type', 'hostid', 'groupids'],
 'srcsinkobject': ['type',
  'filedescriptor',
  'baseobject_hostid',
  'baseobject_permission',
  'baseobject_epoch'],
 'subject': ['hostid',
  'unitid',
  'iteration',
  'count',
  'cmdline',
  'privilegelevel',
  'importedlibraries',
  'exportedlibraries',
  'properties_map_host',
  'parentsubject',
  'type'],
 'unnamedpipeobject': ['sourcefiledescriptor',
  'sinkfiledescriptor',
  'baseobject_hostid',
  'baseobject_per

In [51]:
# generate sql
# template: alter table drop column x, drop column y, ...;
for table, cols in useless_cols.items():
    if cols:
        print(f'alter table {table} drop column {", drop column ".join(cols)};')

alter table event drop column predicateobject2path, drop column predicateobjectpath, drop column size, drop column predicateobject2, drop column predicateobject, drop column subject, drop column name, drop column parameters, drop column hostid, drop column location, drop column programpoint, drop column properties_map_host;
alter table fileobject drop column filedescriptor, drop column localprincipal, drop column size, drop column peinfo, drop column hashes, drop column baseobject_hostid, drop column baseobject_permission, drop column baseobject_epoch;
alter table netflowobject drop column ipprotocol, drop column filedescriptor, drop column baseobject_hostid, drop column baseobject_permission, drop column baseobject_epoch;
alter table principal drop column type, drop column hostid, drop column groupids;
alter table srcsinkobject drop column type, drop column filedescriptor, drop column baseobject_hostid, drop column baseobject_permission, drop column baseobject_epoch;
alter table subje

## cleanup

In [52]:
# cleanup
cur.close()
conn.close()